#### Hypothesis space
The hypothesis space represents all possible fairnesses of the coin where 0 means 100% tail biased and 1 100% head-biased.

#### Calculating likelihoods
**Non flawed machine:**
For a non-flawed machine we can trust the outcome of what it sees, so the likelihoods are as follows:
* $H_l: hs$; fliping heads agrees with the hypothesis space
* $T_l: 1-hs$; flipping tails is against the hypothesis space

**Full flawed machine**
For a full flawed machine is the other way round. This means that when the machine sees a head, it's indeed a tail:
* $H_l: 1-hs$; fliping heads is against the hypothesis space (as we actually flipped a tail, not a head)
* $T_l: hs$; flipping tails agrees with the hypothesis space

**Half-flawed machine**
For a half-flawed machine no matter the outcome of the flip, half of the hypothesis space aggrees with the result and the other half disagrees:
* $H_l: .5\cdot hs + .5\cdot(1-hs)$
* $T_l: .5\cdot hs + .5\cdot(1-hs)$    

This, effectively, bring us back to the starting point where every hypothesis is equally probable. We are substituting the randomness of a coin by the randomness of the machine.

**Intermediate flaw levels**
For intermediate flaw values we have a mix in both quantities, this is for a 1/5 flaw:
* $H_l: .8\cdot hs + .2\cdot(1-hs)$
* $T_l: .2\cdot hs + .8\cdot(1-hs)$

**General expression**  
Regarding all above we can come up with a common expression for all the cases  
$H_l = (1-f)\cdot hs + f(1-hs)$  
$T_l = f\cdot hs + (1-f)\cdot(1-hs)$

In [ ]:
# Solution goes here
from empiricaldist import Pmf
import numpy as np
import seaborn as sns

# The hypothesis space represents all possible fairnesses of the coin
# Where 0 means 100% tail biased and 1 100% head-biased
hs = np.linspace(0, 1, 101)

def calc_posterior(hs, dataset, flaw=0):
    """
    Calculates the posterior probability.
    
    Args:
        hs: numpy array with the hypothesis space
        dataset: a string with the outcome of the experiment.
        flaw: a float that indicates how flawed is the machine.
    returns: Pmf with the posterior probability for the given args.
    """
    prior = Pmf(1, hs)
    assert flaw >= 0 and flaw <= 1
    
    likelihood = {
        'H': (1-flaw) * hs + flaw * (1-hs),
        'T': flaw * hs + (1-flaw) * (1-hs)
    }

    posterior = prior.copy()

    for flip in dataset:
        posterior *= likelihood[flip]

    posterior.normalize()
    return posterior

dataset = 140 * 'H' + 110 * 'T'
y0 = calc_posterior(hs, dataset)
y2 = calc_posterior(hs, dataset, flaw=.2)
y4 = calc_posterior(hs, dataset, flaw=.4)
y5 = calc_posterior(hs, dataset, flaw=.5)

sns.lineplot(x=hs, y=y0, label='y0');
sns.lineplot(x=hs, y=y2, label='y=.2');
sns.lineplot(x=hs, y=y4, label='y=.4');
sns.lineplot(x=hs, y=y5, label='y=.5');